In [1]:
#!/usr/bin/env python
'''Project: Towards the development of a simple fingerprint representation for
the efficient comparison and clustering of large datasets of RNA sequences
By: Abdulvahab Kharadi
First supervisor Name: Irilenia Nobeli
Date: 01/06/20
version: v2.0.0'''

# importing required module
from sys import argv
import cgi
#import cgitb
import hashlib
import re
import os
#cgitb.enable()

def get_data(temp_out = 'temp.out'):
    refine_data = []
    with open (temp_out, 'r') as f:
        lines = f.read().splitlines()
        #print(lines)
        if lines:     
            seq_name = lines.pop(0)
        #print(seq_name)
            seq_name=re.sub(".fa\s\sFORWARD","",re.sub(">","",seq_name))
       
        #seq_name=re.sub(".fa\s\sFORWARD","",seq_name)
        #print(seq_name)
        #s = re.match('^(\>)(\w+\.\w+\.*)(\.fa.+)',seq_name)
        #seq_name = s.groups(0)
            data = [line.split() for line in lines]
            refine_data = [d for d in data if d[0] != 'None' and float(d[0]) < 0]
            return seq_name, refine_data

def process_fa(temp_fa, temp_out="temp.out"):
        os.system("transterm/2ndscore --no-rvs {} > {}".format(temp_fa, temp_out))
        #os.system("rm {}".format(temp_out))

def palindromes(data):
    '''This function take data from a file and return a list of palindromes.
    Each palindrome represented by a four cordinate, two for left steam and two for right steam'''
    palindrome= []
    for i in range(1,len(data)):
        word = data[i]
        energy = float(word[0])
        start = int(word[1])
        lsteam = re.sub('-','',word[5])           #Left steam of a pallindrome,cleaning data
        rsteam = re.sub('-','',word[7])           #Right steam of a pallindrome,cleaning data
        loop = word[6]
        length = int(word[3])
        p = [start,start+len(lsteam)-1,length-len(rsteam)+1,length]
        palindrome.append(p)
    return sorted(palindrome)                     # sorting list accrding to start position

In [2]:
def relation(p1,p2):
    '''This function take list of palindromes and return relationship between two palindromes in form of a Letter.
    Where O=overlapping,S=serial,X=exclusive,I=Included'''
    if p1[1] < p2[0] and p1[2] > p2[1] and p1[3] < p2[2]:
        return 'O'
    elif p1[1]<p2[0] and p1[2] > p2[3] :
        return 'I'
    elif p1[3] < p2[0]:
        return 'S'
    else:
        return 'X'

In [3]:
def pgraph(palindrome):
    '''This function give graph from each pallindrome (consider as Node) to another pallindrome.
    There is no path between pallindrome having Exclusive(X) relation.
    Rerurns dictionary which has Node as key and list of paths from that node as list'''
    #palindrome = palindromes()
    #print(len(palindrome))
    graph = {}
    for i in range(len(palindrome)):
        path = []
        for j in range(i,len(palindrome)):
            r = relation(palindrome[i],palindrome[j])
            if r != 'X':
                path.append(j)
                graph[i] = path
            else:
                graph[i] = path
    print(graph)
    return graph

In [4]:
def path(graph,start,p=[]):
    '''This function give complete paths from each pallindrome'''
    p = p + [start]
    if graph[start] == []:
        return [p]
    paths = []
    for node in graph[start]:
        if node not in p:
            new_paths = path(graph,node,p)
            for new_path in new_paths:
                paths.append(new_path)
    return paths

In [5]:
def dag(paths):
    for p in sorted(paths):
        for j in range(len(paths)-1):
            if set(p) < set(paths[j]):
                try:
                    paths.remove(p)
                    #da_graph = paths
                except:
                    next
                    continue
    return max(paths, key=len)

In [6]:
def fingerprint(da_graph,palindrome):
    '''This function provides a fingerprint for path from each node. We use hashlib to convert
    character presenting relation into a hash number'''
    paths_fingerprint = []
    i =0
    fp = ''
    while i<len(da_graph)-1:
        fp += relation(palindrome[i],palindrome[i+1])
        i += 1
    bfp = fp.encode('utf-8')                            #turn string into byte string
    hfp = hashlib.sha256(bfp)                           #turn bytes into hash object
    index= int(hfp.hexdigest(), 16) % 10**3
    paths_fingerprint.append(index)
    return paths_fingerprint

In [7]:
def tanimoto(s1, s2):
    s12 = set(s1).intersection(set(s2))
    tan = len(s12)/(len(s1) + len(s2) - len(s12))
    return tan

In [8]:
def accuracy(tan):
    if tan >= 0.8 :
        return tan80
    elif tan >= 0.6 and tan < 0.8:
        return tan60
    elif tan >= 0.4 and tan < 0.6:
        return tan40
    elif tan >= 0.2 and tan < 0.4:
        return tan20
    else:
        return tanbad

In [9]:
def run(filename=argv[1],result= {}):
    with open (filename,'r') as f:
        sequences = f.read().splitlines()
    i = 0
    while i < len(sequences):
        deal_single_seq(sequences,i)
        (seq_name, refine_data) = get_data()
        palindrome = palindromes(refine_data)
        graph = pgraph(palindrome)
        paths = path(graph,0)
        da_graph = dag(paths)
        paths_fingerprint = fingerprint(da_graph,palindrome)
        result[seq_name] = paths_fingerprint
        i += 2
    return result

In [10]:
def run_new(dir,result={}):
    for file in os.listdir(dir):
        temp_fa=f'{dir}/{file}'
        process_fa(temp_fa)
        (seq_name, refine_data) = get_data()
        print("1st and 2nd task completed: data_mining model")
        palindrome = palindromes(refine_data)
        print("palindrome function completed")
        graph = pgraph(palindrome)
        '''graph = {0:[1,2,3,7,9],
                1:[4,7],
                2:[4,5,8],
                3:[8],
                4:[],
                5:[8],
                6:[],
                7:[],
                8:[],
                9:[]}'''
        print("graph function completed")
        paths = path(graph,0)
        print("paths generated")
        da_graph = dag(paths)
        print("dag completed")
        paths_fingerprint = fingerprint(da_graph,palindrome)
        result[seq_name] = paths_fingerprint
        print("fingerprint generated")
    return result

In [11]:
path = r"C:\Users\rober\OneDrive\Desktop\MSc_Bioinformatics\Thesis\RNAid-develop\RNAid-develop\hairpins_noreverse.out"
f = open(r"C:\Users\rober\OneDrive\Desktop\MSc_Bioinformatics\Thesis\RNAid-develop\RNAid-develop\hairpins_noreverse.out", 'r')
linesss = f.read().splitlines()
linesss.pop(0)
print(linesss)
# my_lines = [line.split() for line in linesss]
# print(my_lines)

[' 3.9       1 ..      11   xxxxxxxxxxxxxxx                -ATG AAAG CATT                 CTGGCGTAACGCCGC', ' 3.9       2 ..      12   xxxxxxxxxxxxxxA                TGAA AGCA TTC-                 TGGCGTAACGCCGCG', ' 1.4       2 ..      13   xxxxxxxxxxxxxxA                TGAA AGCA TTCT                 GGCGTAACGCCGCGT', ' 4.9       1 ..      14   xxxxxxxxxxxxxxx               ATGAA AGCA TTCTG                GCGTAACGCCGCGTT', ' 5.8       4 ..      15   xxxxxxxxxxxxATG                AAAG CATT CTGG                 CGTAACGCCGCGTTG', ' 3.5       3 ..      16   xxxxxxxxxxxxxAT               GAAAG CATT CTGGC                GTAACGCCGCGTTGC', ' 2.9       6 ..      17   xxxxxxxxxxATGAA                AGC ATTCTG GCG                 TAACGCCGCGTTGCT', '   2       5 ..      18   xxxxxxxxxxxATGA               AAGC ATTCTG GCGT                AACGCCGCGTTGCTC', ' 5.5       4 ..      19   xxxxxxxxxxxxATG              AAAGC ATTCTG GCGTA               ACGCCGCGTTGCTCG', ' 3.2      10 ..      20   xxxxxxATG

In [12]:
def process_fa(temp_fa, temp_out):
        os.system("../transterm/2ndscore --no-rvs {} > {}".format(temp_fa, temp_out))
        #os.system("rm {}".format(temp_out))
process_fa('./hairpins.fna', './my_hairpins.txt')

In [29]:
def get_data(temp_out):
    ## empty list
    refine_data = []
    ## opening hairpins.out file obtained
    ## from 2ndscore
    with open (temp_out, 'r') as f:
        ## creating list off each line in .out file
        lines = f.read().splitlines()
        #print(lines)
        ## extracting first line which corresponds
        ## to identifier of the sequence
        if lines:     
            seq_name = lines.pop(0)
            #print(seq_name)
        ## eliminating the fasta identifier '>'
            seq_name=re.sub(">","",seq_name)
        #seq_name=re.sub(".fa\s\sFORWARD","",seq_name)
        #print(seq_name)
        #s = re.match('^(\>)(\w+\.\w+\.*)(\.fa.+)',seq_name)
        #seq_name = s.groups(0)
        ## data separates the file by a list of lists, where each item is separated
        ## by the empty spaces, converting each item into a list of strings
            data = [line.split() for line in lines]
        ## refine data curates lines with a positive score
            refine_data = [d for d in data if d[0] != 'None' and float(d[0]) > 0]
            return   seq_name, refine_data
#print(get_data('../hairpins_noreverse.out'))
refined_data = get_data('../hairpins_noreverse.out')[0]
print(refined_data)
#print(palindromes(refined_data))

NC_000913.3:c645003-644197 rna [organism=Escherichia coli str. K-12 substr. MG1655] [GeneID=949065] [chromosome=] FORWARD


In [15]:
def palindromes(data):
    '''This function takes the refined data from a get_data and returns a list of palindromes.
    Each palindrome represented by a four cordinate, two for left steam and two for right steam'''
    ## creates empty list
    palindrome= []
    ## itirates through the refined data
    for i in range(1,len(data)):
        ## word divides each line in a list
        word = data[i]
        ## energy hasn't been used
        energy = float(word[0])
        ## start is the base where hairpin starts
        start = int(word[1])
        ## positions 5 and 7 tend to have -
        ## here we remove them left being 5, and right 7
        lsteam = re.sub('-','',word[5])           #Left steam of a pallindrome,cleaning data
        rsteam = re.sub('-','',word[7])           #Right steam of a pallindrome,cleaning data
        ## loop is apparently not used
        loop = word[6]
        ## index 3 represents the lenght of the hairpin
        length = int(word[3])
        ## let's figure why is this sequence of characters a palindrome
        p = [start,start+len(lsteam)-1,length-len(rsteam)+1,length]
        palindrome.append(p)
    return sorted(palindrome)                     # sorting list accrding to start position
print(palindromes(refined_data))
palindrome = palindromes(refined_data)
graph_data = palindromes(refined_data)

[[1, 5, 10, 14], [2, 5, 10, 12], [2, 5, 10, 13], [3, 7, 12, 16], [4, 7, 12, 15], [4, 8, 15, 19], [5, 8, 15, 18], [6, 8, 15, 17], [7, 11, 19, 23], [8, 11, 19, 22], [9, 11, 19, 21], [10, 11, 19, 20], [14, 27, 33, 44], [17, 27, 33, 43], [19, 27, 33, 42], [32, 37, 43, 48], [33, 37, 43, 47], [34, 38, 45, 49], [35, 37, 50, 52], [36, 37, 49, 50], [36, 37, 50, 51], [43, 45, 51, 53], [44, 45, 54, 55], [45, 45, 56, 56], [45, 45, 59, 59], [50, 51, 61, 62], [51, 51, 61, 61], [52, 54, 61, 63], [52, 56, 61, 65], [53, 56, 61, 64], [56, 58, 64, 66], [56, 58, 65, 67], [58, 59, 67, 68], [59, 62, 67, 69]]


In [15]:
def relation(p1,p2):
    '''This function takes the list of palindromes and return relationship between two palindromes in form of a Letter.
    Where O=overlapping,S=serial,X=exclusive,I=Included'''
    if p1[1] < p2[0] and p1[2] > p2[1] and p1[3] < p2[2]:
        return 'O'
    elif p1[1]<p2[0] and p1[2] > p2[3] :
        return 'I'
    elif p1[3] < p2[0]:
        return 'S'
    else:
        return 'X'

def pgraph(palindrome):
    '''This function give graph from each pallindrome (consider as Node) to another pallindrome.
    There is no path between pallindrome having Exclusive(X) relation.
    Rerurns dictionary which has Node as key and list of paths from that node as list'''
    #palindrome = palindromes()
    #print(len(palindrome))
    graph = {}
    ## itirates through each element in the palindromes list
    ## note that palindromes is a list of lists
    for i in range(len(palindrome)):
        ## empty list for each list in palindrome
        path = []
        ## itirates through palindome iteration
        for j in range(i,len(palindrome)):
            r = relation(palindrome[i],palindrome[j])
            if r != 'X':
                path.append(j)
                graph[i] = path
            else:
                graph[i] = path
    return graph
test_graph = pgraph(graph_data)

In [16]:
def path(graph,start,p=[]):
    '''This function give complete paths from each pallindrome'''
    p = p + [start]
    if graph[start] == []:
        return [p]
    paths = []
    for node in graph[start]:
        if node not in p:
            new_paths = path(graph,node,p)
            for new_path in new_paths:
                paths.append(new_path)
    return paths
test_path = path(test_graph, 0)
print(test_path)

[[0, 7, 10, 15, 25, 30, 33], [0, 7, 10, 15, 25, 31], [0, 7, 10, 15, 25, 32], [0, 7, 10, 15, 26, 30, 33], [0, 7, 10, 15, 26, 31], [0, 7, 10, 15, 26, 32], [0, 7, 10, 15, 27, 30, 33], [0, 7, 10, 15, 27, 31], [0, 7, 10, 15, 27, 32], [0, 7, 10, 15, 28, 32], [0, 7, 10, 15, 29, 32], [0, 7, 10, 15, 30, 33], [0, 7, 10, 15, 31], [0, 7, 10, 15, 32], [0, 7, 10, 15, 33], [0, 7, 10, 16, 25, 30, 33], [0, 7, 10, 16, 25, 31], [0, 7, 10, 16, 25, 32], [0, 7, 10, 16, 26, 30, 33], [0, 7, 10, 16, 26, 31], [0, 7, 10, 16, 26, 32], [0, 7, 10, 16, 27, 30, 33], [0, 7, 10, 16, 27, 31], [0, 7, 10, 16, 27, 32], [0, 7, 10, 16, 28, 32], [0, 7, 10, 16, 29, 32], [0, 7, 10, 16, 30, 33], [0, 7, 10, 16, 31], [0, 7, 10, 16, 32], [0, 7, 10, 16, 33], [0, 7, 10, 17, 25, 30, 33], [0, 7, 10, 17, 25, 31], [0, 7, 10, 17, 25, 32], [0, 7, 10, 17, 26, 30, 33], [0, 7, 10, 17, 26, 31], [0, 7, 10, 17, 26, 32], [0, 7, 10, 17, 27, 30, 33], [0, 7, 10, 17, 27, 31], [0, 7, 10, 17, 27, 32], [0, 7, 10, 17, 28, 32], [0, 7, 10, 17, 29, 32], [0,

In [17]:
dagged_data = dag(test_path)
print(dagged_data)

[0, 7, 10, 18, 22, 25, 30, 33]


In [18]:
fingerprints = fingerprint(dagged_data, palindrome)
print(fingerprints)

[253]


In [ ]:
# shuffle sequences from rfam family
# to introduce randomness 

In [4]:
type(os.listdir('../../../Dataset/16S/'))

list

In [ ]:
## creation of fake families, get a distribution of how non-meaninful data would look like.
## 